In [ ]:
import json
from websockets import connect
import websocket
import pandas as pd 

In [ ]:
# !pip install websocket-client

In [ ]:
# !pip install websocket

In [ ]:
# # Replace with your actual API key
# # API_KEY = "YOUR_API_KEY"

# async def get_candlestick_data():
#     uri = "wss://coinswitch.co/api/ws"  # WebSocket endpoint URL
#     async with connect(uri) as websocket:
#         # Subscribe to the channel for the desired currency pair
#         await websocket.send(json.dumps({"setOp": "subscribe", "args": ["spot-btcusdt"]}))

#         while True:
#             try:
#                 data = await websocket.recv()
#                 # Parse JSON data
#                 parsed_data = json.loads(data)

#                 if "k" in parsed_data:  # Check for candlestick data
#                     # Extract relevant information (replace with specific keys you need)
#                     open_price = parsed_data["k"]["o"]
#                     high_price = parsed_data["k"]["h"]
#                     low_price = parsed_data["k"]["l"]
#                     close_price = parsed_data["k"]["c"]
#                     volume = parsed_da["k"]["v"]

#                     # Process the data (e.g., print, store in database)
#                     print(f"Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

#             except Exception as e:
#                 print(f"Error: {e}")
#                 break  # Handle errors and disconnect if needed

# if __name__ == "__main__":
#     import asyncio
#     asyncio.get_event_loop().run_until_complete(get_candlestick_data())


In [ ]:
# import json
# import websocket

# assets = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']

# assets = [coin.lower() + '@kline_1m' for coin in assets]
# assets = '/'.join(assets)

# def on_message(ws, message):
#     message = json.loads(message)
#     print(message)

# def on_error(ws, error):
#     print(error)

# def on_close(ws):
#     print("### closed ###")

# def on_open(ws):
#     print("### connected ###")

# socket = "wss://stream.binance.com:9443/stream?streams=" + assets
# ws = websocket.WebSocketApp(socket,
#                             on_message=on_message,
#                             on_error=on_error,
#                             on_close=on_close)
# ws.on_open = on_open

# ws.run_forever()
